Being more comfortable with Python language than Scala, I decided to fire up the included pyspask shell, but I personally don't find it engaging. To get a better grip on how everything works, I decided to work with Jupyter notebook.

*The Jupyter Notebook is a web application that allows you to create and share documents that contain live code, equations, visualizations and explanatory text. Uses include: data cleaning and transformation, numerical simulation, statistical modeling, machine learning and much more*

Jupyter lets me program like on a shell, but has the longetivity of written code. I **reaaaally** like this interface :)

In [1]:
pwd

u'/vagrant/ipynb/FSProject'

In [14]:
ls -l

total 10
-rwxrwxrwx 1 vagrant vagrant 1510 2016-12-02 05:19 derby.log*
drwxrwxrwx 1 vagrant vagrant    0 2016-12-01 22:27 scripts/
drwxrwxrwx 1 vagrant vagrant    0 2016-12-01 21:11 sfexample/
-rwxrwxrwx 1 vagrant vagrant 8045 2016-12-02 05:44 Sf_Notebook.ipynb*


### Step 1

* Create txt file with data and upload it to HDFS

In [45]:
cat sfexample/sensors/sensors_data.txt

### Step 2
* Create HQL script to create the table and map it to the data on HDFS
* Save it to the local drive in my virtual machine
* Run script: **hive -f create_sensors_table.hql **

In [41]:
cd /vagrant/ipynb/FSProject

/vagrant/ipynb/FSProject


In [42]:
ls -l

total 13
-rwxrwxrwx 1 vagrant vagrant  1510 2016-12-02 05:19 derby.log*
drwxrwxrwx 1 vagrant vagrant     0 2016-12-01 22:27 scripts/
drwxrwxrwx 1 vagrant vagrant     0 2016-12-01 21:11 sfexample/
-rwxrwxrwx 1 vagrant vagrant 11133 2016-12-02 06:10 Sf_Notebook.ipynb*


In [43]:
cat scripts/create_sensors_table.hql

USE sfexample;

CREATE EXTERNAL TABLE IF NOT EXISTS Sensors (
Gateway_SN string,
Model_Number string,
Alarm_Number int )
ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION "hdfs:///user//vagrant//sfexample//sensors//";



### Step 3 
* Time to play with the data

First step, in any Apache programming is to create a SparkContext. SparkContext is required when we want to execute operations in a cluster. SparkContext tells Spark how and where to access a cluster. And the first step is to connect with Apache Cluster. If you are using Spark Shell, you will notice that it is already created. Otherwise, we can create the SparkContext by importing, initializing and providing the configuration settings

In [3]:
from pyspark import SparkContext
sqlContext = SQLContext(sc)


In [5]:
sqlctx = HiveContext(sc)
sqlctx.sql("USE sfexample")
sqlctx.tableNames() #

[u'sensors']

In [63]:
sensors = sqlctx.table("sensors")
sensors.show(20)

+----------+------------+------------+
|gateway_sn|model_number|alarm_number|
+----------+------------+------------+
|0112D00003| 5810L120_07|         403|
|0112D00003| 5810L120_08|         301|
|0112D00003| 5810L120_07|         122|
|0112D00003| 5810L120_07|         403|
|0112D00003| 5810L120_07|         403|
|0112D00003| 5810L120_07|         403|
|0112D00003| 5810L120_07|         301|
|0112D00003| 5810L120_08|         301|
|0112D00003| 5810L120_08|         301|
|0112D00003| 5810L120_08|         203|
|0112D00003| 5810L120_08|         203|
|0112D00003| 5810L120_06|         403|
|0112D00003| 5810L120_06|         403|
|0112D00003| 5810L120_06|         403|
|0112D00003| 5810L120_06|         403|
|0112D00003| 5810L120_06|         155|
|0112D00003| 5810L120_05|         211|
|0112D00003| 5810L120_05|         211|
|0112D00003| 5810L120_05|         211|
|0112D00003| 5810L120_05|         211|
+----------+------------+------------+
only showing top 20 rows



In [64]:
sensors.printSchema()

root
 |-- gateway_sn: string (nullable = true)
 |-- model_number: string (nullable = true)
 |-- alarm_number: integer (nullable = true)



In [65]:
sensors.count()

22

In [8]:
revnumresults = "SELECT SUBSTR(model_number, -2) AS Revision_Number, Alarm_Number FROM sensors"

In [9]:
revnumresults_df = sqlctx.sql(revnumresults)

In [10]:
revnumresults_df.show()

+---------------+------------+
|Revision_Number|Alarm_Number|
+---------------+------------+
|             07|         403|
|             08|         301|
|             07|         122|
|             07|         403|
|             07|         403|
|             07|         403|
|             07|         301|
|             08|         301|
|             08|         301|
|             08|         203|
|             08|         203|
|             06|         403|
|             06|         403|
|             06|         403|
|             06|         403|
|             06|         155|
|             05|         211|
|             05|         211|
|             05|         211|
|             05|         211|
+---------------+------------+
only showing top 20 rows



In [11]:
revnumresults_df.groupby('Revision_Number').count().sort('Revision_Number').show()

+---------------+-----+
|Revision_Number|count|
+---------------+-----+
|             05|    4|
|             06|    5|
|             07|    6|
|             08|    6|
|             09|    1|
+---------------+-----+



In [12]:
revnumresults_df.groupby('Revision_Number','Alarm_Number').count().sort('Revision_Number').show()

+---------------+------------+-----+
|Revision_Number|Alarm_Number|count|
+---------------+------------+-----+
|             05|         211|    4|
|             06|         403|    4|
|             06|         155|    1|
|             07|         122|    1|
|             07|         403|    4|
|             07|         301|    1|
|             08|         301|    3|
|             08|         203|    3|
|             09|         212|    1|
+---------------+------------+-----+



In [47]:
revnumresults_df.printSchema()

root
 |-- Revision_Number: string (nullable = true)
 |-- Alarm_Number: integer (nullable = true)



### Step 4
* Create python script and submit job: **spark-submit revision_number_analysis.py**

In [66]:
cat revision_number_analysis.py

## Spark Application - execute with spark-submit
## Imports

from pyspark import SparkConf, SparkContext, SQLContext, HiveContext

APP_NAME = 'REVISION NUMBER ANALYSIS'

## Main functionality
def main(sc):
    sqlContext = SQLContext(sc)
    sqlctx = HiveContext(sc)
    sqlctx.sql("USE sfexample")
    revnumresults = "SELECT SUBSTR(model_number, -2) AS Revision_Number, Alarm_Number FROM sensors"
    revnumresults_df = sqlctx.sql(revnumresults)
    revnumresults_df.groupby('Revision_Number').count().sort('Revision_Number').show()

if __name__ == "__main__":
    # Configure Spark
    conf = SparkConf().setMaster("local[*]")
    conf = conf.setAppName(APP_NAME)
    sc   = SparkContext(conf=conf)

    # Execute Main functionality
    main(sc)

In [67]:
! spark-submit revision_number_analysis.py

16/12/02 07:14:33 WARN SparkConf: The configuration key 'spark.yarn.applicationMaster.waitTries' has been deprecated as of Spark 1.3 and and may be removed in the future. Please use the new key 'spark.yarn.am.waitTime' instead.
16/12/02 07:14:33 INFO SparkContext: Running Spark version 1.6.0
16/12/02 07:14:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/12/02 07:14:34 WARN SparkConf: The configuration key 'spark.yarn.applicationMaster.waitTries' has been deprecated as of Spark 1.3 and and may be removed in the future. Please use the new key 'spark.yarn.am.waitTime' instead.
16/12/02 07:14:34 INFO SecurityManager: Changing view acls to: vagrant
16/12/02 07:14:34 INFO SecurityManager: Changing modify acls to: vagrant
16/12/02 07:14:34 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users with view permissions: Set(vagrant); users with modify permissions: Set(vagrant)
16/12